In [1]:
import telebot
import pandas as pd
from machine_state import MachineStates, Test
from preprocessing import clear_df
from catboost import CatBoostClassifier

In [2]:
model = CatBoostClassifier()
model.load_model('../model/model')

# Опросник для титиника
step_by_step =(
    (
        'В каком классе обслуживания вы бы поехали?',
        Test( int_test=True, boundaries_test=True, sequence_boundaries=(1,3) )),
    (
        'Какое у Вас имя на английском? (ФИО)',
        Test(valid_text_test=True, regex_for_valid_text='^\s*\w+\s+\w+\s+\w+\s*$')),
    (
        'Какой у Вас пол? (male/female)', 
        Test(entering_test=True, entering_set=('male', 'female'))),
    (
        'Сколько Вам лет?', 
        Test(boundaries_test=True, sequence_boundaries=(0, 100))),
    (
        'Сколько с Вами поедет сестер, братьев?', 
        Test(int_test=True, boundaries_test=True, sequence_boundaries=(0, 30))),
    (
        'Сколько с Вами поедет родителей и Ваших детей?',
        Test(int_test=True, boundaries_test=True, sequence_boundaries=(0, 30))),
    (
        'Какой номер у Вашего билета? (5 цифр) ',
        Test(valid_text_test=True, regex_for_valid_text='^\s*\d{5}\s*$')),
    (
        'Какая у Вас кабина? (3 цифры) ',
        Test(valid_text_test=True, regex_for_valid_text='^\s*\d{3}\s*$')
    ),
    (
        'Сколько стоит Ваш билет?',
        Test(boundaries_test=True, sequence_boundaries=(0, 500))),
    (
        'В каком городе Вы бы зашли на борт Титаника?',
        Test(entering_test=True, entering_set=('Q', 'S', 'C'))),
    (
        'Поздравляю, Вы прошли опрос!', 
        None)
)

COLUMNS = [
    'PassengerId',
    'Pclass',
    'Name',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Ticket',
    'Fare',
    'Cabin',
    'Embarked'
]
# типы данных для колонок
DTYPES = pd.read_csv('../data/titanic/test.csv').dtypes

In [5]:
def set_types_df(df):
    for col in DTYPES.index:
        df[col] = df[col].astype(DTYPES.at[col])
        
    return df


def get_submit(arr):
    if len(arr) < 11:
        arr = [0] + arr
        
    df_temp = pd.DataFrame(arr).T
    df_temp.columns = COLUMNS
    df_temp = set_types_df(df_temp)
    df_temp = clear_df(df_temp)
    return 'К сожалению, Вы бы не выжили' if model.predict(df_temp.values)[0] == 0 else 'Поздравляем, Вы бы выжили!'


In [8]:

IDS_machine_state = {}

TOKEN = open('../TOKEN.txt').read()

#Сюда подставляете свой токен
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def start_message(message):
    global IDS_machine_state
    
    # создание машины состояний для пользователя
    IDS_machine_state[message.chat.id] = MachineStates(step_by_step)
    
    msg = IDS_machine_state[message.chat](answer=None)
    print(type(msg))
    bot.send_message(message.chat.id, f'Выжили бы Вы на Титанике?\n\n{msg}')
    # создание машины состояний для пользователя
    #IDS_machine_state[message.chat.id] = MachineStates(step_by_step)
    # отправка первого вопроса
    #bot.send_message(message.chat.id, IDS_machine_state[message.chat]()[0])


@bot.message_handler(commands=['last'])
def last_message(message):
    if message.chat.id in IDS_machine_state:
        # переход на предыдущий шаг
        machine_reaction = IDS_machine_state[message.chat.id](get_last=True)
        # отправка текста
        bot.send_message(message.chat.id, machine_reaction[0])
    

@bot.message_handler(content_types=['text'])
def send_text(message='text'):
    global IDS_machine_state
    # id чата
    chat_id = message.chat.id 
    
    if chat_id not in IDS_machine_state:
        IDS_machine_state[chat_id] = MachineStates(step_by_step)

    # Вызов машины состояния
    machine_reaction = IDS_machine_state[chat_id](answer=message.text)
    # отправляем сообщение об окончании
    bot.send_message(message.chat.id, machine_reaction[0])
    # если достигнут последний вопрос
    if machine_reaction[1]:
        # отправляем на предсказание
        submit = get_submit(IDS_machine_state[chat_id].get_answer())
        # отправляем результат
        bot.send_message(message.chat.id, str(submit))
        # пересоздаем машину состаяния
        IDS_machine_state[chat_id] = MachineStates(step_by_step)
    

bot.polling()
#сами смайлики прописать не могу, иначе публикация не обрабатывается на habr